# Recursive Retriever + Document Agents

TODO: 

In [1]:
from llama_index import (
    VectorStoreIndex,
    ListIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
)
from llama_index.schema import IndexNode
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.llms import OpenAI
from llama_index.callbacks import CallbackManager

In [2]:
wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Houston"]

In [3]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [4]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

Define LLM + Service Context + Callback Manager

In [5]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
callback_manager = CallbackManager([])
# TMP
callback_manager.id = "testing"

service_context = ServiceContext.from_defaults(llm=llm, callback_manager=callback_manager)

INITIALIZING CALLBACK MANAGER
INITIALIZING CALLBACK MANAGER
INITIALIZING CALLBACK MANAGER


## Build Document Agent for each Document

This document agent can dynamically choose to perform semantic search or summarization within a given document.

We create a separate document agent for each city.

In [6]:
from llama_index.agent import OpenAIAgent

# Build agents dictionary
agents = {}

for wiki_title in wiki_titles:
    # build vector index
    vector_index = VectorStoreIndex.from_documents(
        city_docs[wiki_title], service_context=service_context
    )
    # build list index
    list_index = ListIndex.from_documents(
        city_docs[wiki_title], service_context=service_context
    )
    # define query engines
    vector_query_engine = vector_index.as_query_engine(callback_manager=callback_manager)
    list_query_engine = list_index.as_query_engine(callback_manager=callback_manager)
    
    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=f"Useful for summarization questions related to {wiki_title}"
            ),
        ),
        QueryEngineTool(
            query_engine=list_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=f"Useful for retrieving specific context from {wiki_title}"
            ),
        ),
    ]
    
    # build agent
    function_llm = OpenAI(model="gpt-3.5-turbo-0613")
    agent = OpenAIAgent.from_tools(
        query_engine_tools, llm=function_llm, verbose=True,
        callback_manager=callback_manager
    )
    
    agents[wiki_title] = agent    
    
    

STARTING TRACE: ('index_construction', [])
on event start, global trace stack: ['root', '60590c9b-6b8d-469e-a932-ee45b9c2c7d6']
on event end, global trace stack: ['root']
STARTING TRACE: ('index_construction', ['index_construction'])
ENDING TRACE: ('index_construction', ['index_construction', 'index_construction'])
ENDING TRACE: ('index_construction', ['index_construction'])
STARTING TRACE: ('index_construction', [])
on event start, global trace stack: ['root', 'ee12d516-dd2a-4088-80b1-a3fc28d16443']
on event end, global trace stack: ['root']
STARTING TRACE: ('index_construction', ['index_construction'])
ENDING TRACE: ('index_construction', ['index_construction', 'index_construction'])
ENDING TRACE: ('index_construction', ['index_construction'])
INITIALIZING CALLBACK MANAGER
STARTING TRACE: ('index_construction', [])
on event start, global trace stack: ['root', 'b862aed4-e343-4098-ab12-d19f9e01a2bb']
on event end, global trace stack: ['root']
STARTING TRACE: ('index_construction', ['in

## Build Recursive Retriever over these Agents

In [7]:
# define top-level nodes
nodes = []
for wiki_title in wiki_titles:
    # define index node that links to these agents
    wiki_summary = (
        f"This content contains Wikipedia articles about {wiki_title}. "
        f"Use this index if you need to lookup specific facts about {wiki_title}.\n"
        "Do not use this index if you want to analyze multiple cities."
    )
    node = IndexNode(text=wiki_summary, index_id=wiki_title)
    nodes.append(node)


In [8]:
# define top-level retriever
vector_index = VectorStoreIndex(nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1, callback_manager=callback_manager)

INITIALIZING CALLBACK MANAGER
INITIALIZING CALLBACK MANAGER
INITIALIZING CALLBACK MANAGER
STARTING TRACE: ('index_construction', [])
ENDING TRACE: ('index_construction', ['index_construction'])


In [9]:
# define recursive retriever
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import get_response_synthesizer

In [10]:
# note: can pass `agents` dict as `query_engine_dict` since every agent can be used as a query engine
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=agents,
    verbose=True,
    callback_manager=callback_manager
)

#### Define Full Query Engine 

This query engine uses the recursive retriever + response synthesis module to synthesize a response.

In [14]:
response_synthesizer = get_response_synthesizer(
    # service_context=service_context,
    response_mode="compact",
    callback_manager=callback_manager
)
query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer,
    service_context=service_context
)

INITIALIZING CALLBACK MANAGER
INITIALIZING CALLBACK MANAGER


In [15]:
# agents["Boston"].query("Tell me about the sports teams in Boston")

In [16]:
# should use Boston agent -> vector tool
response = query_engine.query(
    "Tell me about the sports teams in Boston"
)

starting query engine, callback id: testing
STARTING TRACE: ('query', [])
on event start, global trace stack: ['root', '87b4614d-759e-42bc-99ad-cf90848bee2f']
before retrieve: ['root', '87b4614d-759e-42bc-99ad-cf90848bee2f']
on event start, global trace stack: ['root', '87b4614d-759e-42bc-99ad-cf90848bee2f', '036788b9-5e76-4361-9b9c-25a606539542']
Retrieving with query id None: Tell me about the sports teams in Boston
on event start, global trace stack: ['root', '87b4614d-759e-42bc-99ad-cf90848bee2f', '036788b9-5e76-4361-9b9c-25a606539542', '86038214-e6c7-443b-8046-73fc46b922c5']
on event end, global trace stack: ['root', '87b4614d-759e-42bc-99ad-cf90848bee2f', '036788b9-5e76-4361-9b9c-25a606539542']
Retrieved node with id, entering: Boston
Retrieving with query id Boston: Tell me about the sports teams in Boston
starting query engine, callback id: testing
STARTING TRACE: ('query', ['query'])
STARTING TRACE: ('query', ['query', 'query'])
STARTING TRACE: ('chat', ['query', 'query', 'que

In [18]:
print(response)

Boston is home to several professional sports teams in the major North American leagues. The city has teams in MLB, NFL, NBA, and NHL. The Boston Red Sox are a successful baseball team with multiple World Series championships. The New England Patriots are a dominant NFL team with six Super Bowl championships. The Boston Celtics have a rich history in the NBA, winning a record 17 NBA championships. The Boston Bruins are a successful NHL team with six Stanley Cup championships. Additionally, Boston has a Major League Soccer team called the New England Revolution. Overall, Boston has a strong sports culture and its teams have a dedicated fan base.


In [32]:
from llama_index.callbacks.base import global_stack_trace

global_stack_trace

[]

In [ ]:
# should use Houston agent -> vector tool
response = query_engine.query(
    "Tell me about the sports teams in Houston"
)

In [ ]:
# should use Seattle agent -> summary tool 
response = query_engine.query(
    "Tell me about all the positive aspects of Chicago; write a song about it"
)